# Initializing the SMD

In [9]:
# Imports
import config as cf
import utils as ut

import numpy as np
import pandas as pd
import datetime
import pickle

## Connections

In [10]:
# Connect to DB
conn = ut.connect_db()
cursor = conn.cursor()

# Connect to Alpaca API
api = ut.connect_api()

|---------------| Successfully connected to securities_master. |---------------|
|---------------| Successfully connected to Alpaca API. |---------------|


## Write exchanges to SMD

In [ ]:
# Exchanges
sql = """
  INSERT INTO exchange (mic_code, acronym, name, region, country, city, currency) VALUES 
  ('XNAS', 'NASDAQ', 'Nasdaq - All Markets', 'North America', 'United States', 'New York', 'USD'),
  ('XNYS', 'NYSE', 'New York Stock Exchange', 'North America', 'United States', 'New York', 'USD'),
  ('XASE', 'AMEX', 'NYSE American', 'North America', 'United States', 'New York', 'USD'),
  ('ARCX', 'ARCA', 'NYSE Arca', 'North America', 'United States', 'New York', 'USD'),
  ('BATS', 'BATS', 'Bats Global Markets', 'North America', 'United States', 'Chicago', 'USD');"""
cursor.execute(sql)
conn.commit()
print("|---------------| Data successfully written to database. |---------------|")

## Write instruments to SMD

In [ ]:
# Get all assets
assets = api.list_assets()

In [ ]:
# Exchanges
nasdaq = ut.populate_exchange_list('NASDAQ', assets) # 4707
nyse = ut.populate_exchange_list('NYSE', assets) # 2955
amex = ut.populate_exchange_list('AMEX', assets) # 263
arca = ut.populate_exchange_list('ARCA', assets) # 1788
bats = ut.populate_exchange_list('BATS', assets) # 531

In [ ]:
# Cast Tickers to YF-format
nasdaq = sorted(ut.cast_to_yf(nasdaq))
nyse = sorted(ut.cast_to_yf(nyse))
amex = sorted(ut.cast_to_yf(amex))
arca = sorted(ut.cast_to_yf(arca))
bats = sorted(ut.cast_to_yf(bats))

### NASDAQ

In [ ]:
nasdaq_alpaca = ut.populate_exchange_df('NASDAQ', assets, conn)
nasdaq_alpaca

In [ ]:
splits = ut.determine_splits(nasdaq, cf.BATCH_SIZE)
split_arr = np.array_split(nasdaq, splits)
nasdaq_info = pd.DataFrame()

count = 1
for array in split_arr:
  print("|---------------| Split {} of {} |---------------|".format(count, splits))
  tmp = ut.get_corporate_info(array, 50, cf.KOI)
  nasdaq_info = nasdaq_info.append(tmp)
  count += 1

In [ ]:
nasdaq_info

In [ ]:
tmp = ut.cast_to_alpaca(nasdaq_info['symbol'].tolist()) # cast tickers back to Alpaca-format
nasdaq_info.drop('symbol', axis = 1, inplace = True)
nasdaq_info.insert(0, 'ticker', tmp)

In [ ]:
nasdaq_df = pd.merge(nasdaq_alpaca, nasdaq_info, on = 'ticker', how = 'outer')
nasdaq_df = nasdaq_df.sort_values(by = ['ticker'])

In [ ]:
nasdaq_df

In [ ]:
nasdaq_df.to_csv('data/NASDAQ.csv')

### NYSE

In [ ]:
nyse_alpaca = ut.populate_exchange_df('NYSE', assets, conn)
nyse_alpaca

In [ ]:
splits = ut.determine_splits(nyse, cf.BATCH_SIZE)
split_arr = np.array_split(nyse, splits)
nyse_info = pd.DataFrame()

count = 1
for array in split_arr:
  print("|---------------| Split {} of {} |---------------|".format(count, splits))
  tmp = ut.get_corporate_info(array, 50, cf.KOI)
  nyse_info = nyse_info.append(tmp)
  count += 1

In [ ]:
nyse_info

In [ ]:
tmp = ut.cast_to_alpaca(nyse_info['symbol'].tolist())
nyse_info.drop('symbol', axis = 1, inplace = True)
nyse_info.insert(0, 'ticker', tmp)

In [ ]:
nyse_df = pd.merge(nyse_alpaca, nyse_info, on = 'ticker', how = 'outer')
nyse_df = nyse_df.sort_values(by = ['ticker'])

In [ ]:
nyse_df

In [ ]:
nyse_df.to_csv('data/NYSE.csv')

### AMEX

In [ ]:
amex_alpaca = ut.populate_exchange_df('AMEX', assets, conn)
amex_alpaca

In [ ]:
splits = ut.determine_splits(amex, cf.BATCH_SIZE)
split_arr = np.array_split(amex, splits)
amex_info = pd.DataFrame()

count = 1
for array in split_arr:
  print("|---------------| Split {} of {} |---------------|".format(count, splits))
  tmp = ut.get_corporate_info(array, 50, cf.KOI)
  amex_info = amex_info.append(tmp)
  count += 1

In [ ]:
amex_info

In [ ]:
tmp = ut.cast_to_alpaca(amex_info['symbol'].tolist())
amex_info.drop('symbol', axis = 1, inplace = True)
amex_info.insert(0, 'ticker', tmp)

In [ ]:
amex_df = pd.merge(amex_alpaca, amex_info, on = 'ticker', how = 'outer')
amex_df = amex_df.sort_values(by = ['ticker'])

In [ ]:
amex_df

In [ ]:
amex_df.to_csv('data/AMEX.csv')

### ARCA

In [ ]:
arca_alpaca = ut.populate_exchange_df('ARCA', assets, conn)
arca_alpaca

In [ ]:
splits = ut.determine_splits(arca, cf.BATCH_SIZE)
split_arr = np.array_split(arca, splits)
arca_info = pd.DataFrame()

count = 1
for array in split_arr:
  print("|---------------| Split {} of {} |---------------|".format(count, splits))
  tmp = ut.get_corporate_info(array, 50, cf.KOI)
  arca_info = arca_info.append(tmp)
  count += 1

In [ ]:
arca_info

In [ ]:
tmp = ut.cast_to_alpaca(arca_info['symbol'].tolist())
arca_info.drop('symbol', axis = 1, inplace = True)
arca_info.insert(0, 'ticker', tmp)

In [ ]:
arca_df = pd.merge(arca_alpaca, arca_info, on = 'ticker', how = 'outer')
arca_df = arca_df.sort_values(by = ['ticker'])

In [ ]:
arca_df

In [ ]:
arca_df.to_csv('data/ARCA.csv')

### BATS

In [ ]:
bats_alpaca = ut.populate_exchange_df('BATS', assets, conn)
bats_alpaca

In [ ]:
splits = ut.determine_splits(bats, cf.BATCH_SIZE)
split_arr = np.array_split(bats, splits)
bats_info = pd.DataFrame()

count = 1
for array in split_arr:
  print("|---------------| Split {} of {} |---------------|".format(count, splits))
  tmp = ut.get_corporate_info(array, 50, cf.KOI)
  bats_info = bats_info.append(tmp)
  count += 1

In [ ]:
bats_info

In [ ]:
tmp = ut.cast_to_alpaca(bats_info['symbol'].tolist())
bats_info.drop('symbol', axis = 1, inplace = True)
bats_info.insert(0, 'ticker', tmp)

In [ ]:
bats_df = pd.merge(bats_alpaca, bats_info, on = 'ticker', how = 'outer')
bats_df = bats_df.sort_values(by = ['ticker'])

In [ ]:
bats_df

In [ ]:
bats_df.to_csv('data/BATS.csv')

### Actually write to SMD

In [ ]:
nasdaq_df = pd.read_csv('data/NASDAQ.csv')
nasdaq_df = nasdaq_df.loc[:, nasdaq_df.columns != 'Unnamed: 0']
nasdaq_df = nasdaq_df.replace({np.nan: None}) # replace NaN with None for MySQL

nyse_df = pd.read_csv('data/NYSE.csv')
nyse_df = nyse_df.loc[:, nyse_df.columns != 'Unnamed: 0']
nyse_df = nyse_df.replace({np.nan: None})

amex_df = pd.read_csv('data/AMEX.csv')
amex_df = amex_df.loc[:, amex_df.columns != 'Unnamed: 0']
amex_df = amex_df.replace({np.nan: None})

arca_df = pd.read_csv('data/ARCA.csv')
arca_df = arca_df.loc[:, arca_df.columns != 'Unnamed: 0']
arca_df = arca_df.replace({np.nan: None})

bats_df = pd.read_csv('data/BATS.csv')
bats_df = bats_df.loc[:, bats_df.columns != 'Unnamed: 0']
bats_df = bats_df.replace({np.nan: None})

In [ ]:
# Dictionary for easy iterating
exchanges = {
  'NASDAQ': nasdaq_df,
  'NYSE': nyse_df,
  'AMEX': amex_df,
  'ARCA': arca_df,
  'BATS': bats_df,
}

In [ ]:
# Writing to the database
for key, exchange in exchanges.items():
  for row in exchange.itertuples(index = False):
    cursor.execute("INSERT INTO instrument (exchange_id, ticker, name, sector, industry, country, website) VALUES (%s, %s, %s, %s, %s, %s, %s)", row)
conn.commit()
print("|---------------| Data successfully written to database. |---------------|")

## Write vendors to SMD

In [ ]:
# Yahoo Finance
sql = "INSERT INTO vendor (name, website) VALUES ('Yahoo Finance', 'https://finance.yahoo.com/')"
cursor.execute(sql)
conn.commit()
print("|---------------| Data successfully written to database. |---------------|")

In [ ]:
# Alpaca
sql = "INSERT INTO vendor (name, website) VALUES ('Alpaca', 'https://alpaca.markets/')"
cursor.execute(sql)
conn.commit()
print("|---------------| Data successfully written to database. |---------------|")

## Write prices to SMD

In [ ]:
today = datetime.date.today()
yesterday = str(today - datetime.timedelta(days = 1))

In [ ]:
nasdaq_tickers = ut.get_exchange_tickers('NASDAQ', conn)
nyse_tickers = ut.get_exchange_tickers('NYSE', conn)
amex_tickers = ut.get_exchange_tickers('AMEX', conn)
arca_tickers = ut.get_exchange_tickers('ARCA', conn)
bats_tickers = ut.get_exchange_tickers('BATS', conn)

### NASDAQ

In [ ]:
splits = ut.determine_splits(nasdaq_tickers['ticker'].to_list(), cf.BATCH_SIZE)
nasdaq_tickers_list = [nasdaq_tickers.loc[i : i + cf.BATCH_SIZE - 1, : ] for i in range(0, len(nasdaq_tickers), cf.BATCH_SIZE)]
nasdaq_excepts = []

count = 1
for batch in nasdaq_tickers_list:
  print("|---------------| Split {} of {} |---------------|".format(count, splits))
  tmp = ut.write_historical_prices_yf(batch, conn, cursor, None, yesterday)
  nasdaq_excepts.append(tmp)
  count += 1

pickle.dump(ut.flatten_exceptions(nasdaq_excepts), open("data/NASDAQ.txt", "wb")) # save exceptions

In [ ]:
nasdaq_excepts = pickle.load(open("data/NASDAQ.txt", "rb"))
nasdaq_excepts

In [ ]:
nasdaq_alpaca = pd.DataFrame()
for ticker in nasdaq_excepts:
  nasdaq_alpaca = nasdaq_alpaca.append(nasdaq_tickers[nasdaq_tickers['ticker'] == ticker])
ut.write_historical_prices_alpaca(nasdaq_alpaca, api, conn, cursor, None, yesterday)

### NYSE

In [ ]:
splits = ut.determine_splits(nyse_tickers['ticker'].to_list(), cf.BATCH_SIZE)
nyse_tickers_list = [nyse_tickers.loc[i : i + cf.BATCH_SIZE - 1, : ] for i in range(0, len(nyse_tickers), cf.BATCH_SIZE)]
nyse_excepts = []

count = 1
for batch in nyse_tickers_list:
  print("|---------------| Split {} of {} |---------------|".format(count, splits))
  tmp = ut.write_historical_prices_yf(batch, conn, cursor, None, yesterday)
  nyse_excepts.append(tmp)
  count += 1

pickle.dump(ut.flatten_exceptions(nyse_excepts), open("data/NYSE.txt", "wb"))

In [ ]:
nyse_excepts = pickle.load(open("data/NYSE.txt", "rb"))
nyse_excepts

In [ ]:
nyse_alpaca = pd.DataFrame()
for ticker in nyse_excepts:
  nyse_alpaca = nyse_alpaca.append(nyse_tickers[nyse_tickers['ticker'] == ticker])
ut.write_historical_prices_alpaca(nyse_alpaca, api, conn, cursor, None, yesterday)

### AMEX

In [ ]:
splits = ut.determine_splits(amex_tickers['ticker'].to_list(), cf.BATCH_SIZE)
amex_tickers_list = [amex_tickers.loc[i : i + cf.BATCH_SIZE - 1, : ] for i in range(0, len(amex_tickers), cf.BATCH_SIZE)]
amex_excepts = []

count = 1
for batch in amex_tickers_list:
  print("|---------------| Split {} of {} |---------------|".format(count, splits))
  tmp = ut.write_historical_prices_yf(batch, conn, cursor, None, yesterday)
  amex_excepts.append(tmp)
  count += 1

pickle.dump(ut.flatten_exceptions(amex_excepts), open("data/AMEX.txt", "wb"))

In [ ]:
amex_excepts = pickle.load(open("data/AMEX.txt", "rb"))
amex_excepts

In [ ]:
amex_alpaca = pd.DataFrame()
for ticker in amex_excepts:
  amex_alpaca = amex_alpaca.append(amex_tickers[amex_tickers['ticker'] == ticker])
ut.write_historical_prices_alpaca(amex_alpaca, api, conn, cursor, None, yesterday)

### ARCA

In [ ]:
splits = ut.determine_splits(arca_tickers['ticker'].to_list(), cf.BATCH_SIZE)
arca_tickers_list = [arca_tickers.loc[i : i + cf.BATCH_SIZE - 1, : ] for i in range(0, len(arca_tickers), cf.BATCH_SIZE)]
arca_excepts = []

count = 1
for batch in arca_tickers_list:
  print("|---------------| Split {} of {} |---------------|".format(count, splits))
  tmp = ut.write_historical_prices_yf(batch, conn, cursor, None, yesterday)
  arca_excepts.append(tmp)
  count += 1

pickle.dump(ut.flatten_exceptions(arca_excepts), open("data/ARCA.txt", "wb"))

In [ ]:
arca_excepts = pickle.load(open("data/ARCA.txt", "rb"))
arca_excepts

In [ ]:
arca_alpaca = pd.DataFrame()
for ticker in arca_excepts:
  arca_alpaca = arca_alpaca.append(arca_tickers[arca_tickers['ticker'] == ticker])
ut.write_historical_prices_alpaca(arca_alpaca, api, conn, cursor, None, yesterday)

### BATS

In [ ]:
splits = ut.determine_splits(bats_tickers['ticker'].to_list(), cf.BATCH_SIZE)
bats_tickers_list = [bats_tickers.loc[i : i + cf.BATCH_SIZE - 1, : ] for i in range(0, len(bats_tickers), cf.BATCH_SIZE)]
bats_excepts = []

count = 1
for batch in bats_tickers_list:
  print("|---------------| Split {} of {} |---------------|".format(count, splits))
  tmp = ut.write_historical_prices_yf(batch, conn, cursor, None, yesterday)
  bats_excepts.append(tmp)
  count += 1

pickle.dump(ut.flatten_exceptions(bats_excepts), open("data/BATS.txt", "wb"))

In [ ]:
bats_excepts = pickle.load(open("data/BATS.txt", "rb"))
bats_excepts

In [ ]:
bats_alpaca = pd.DataFrame()
for ticker in bats_excepts:
  bats_alpaca = bats_alpaca.append(bats_tickers[bats_tickers['ticker'] == ticker])
ut.write_historical_prices_alpaca(bats_alpaca, api, conn, cursor, None, yesterday)

## Write industries to SMD

```sql
CREATE TABLE IF NOT EXISTS `industry` (
  `id` INT(11) NOT NULL AUTO_INCREMENT,
  `name` VARCHAR(255) NULL,
  `created_date` DATETIME NULL DEFAULT CURRENT_TIMESTAMP(),
  `last_updated` DATETIME NULL DEFAULT CURRENT_TIMESTAMP() ON UPDATE CURRENT_TIMESTAMP(),
  PRIMARY KEY (`id`))
  ENGINE = InnoDB
  DEFAULT CHARACTER SET = utf8;
```

In [8]:
industries = pd.read_sql("SELECT DISTINCT(industry) FROM instrument;", conn)
industries.dropna(inplace = True)
industries.sort_values(by = 'industry', inplace = True)
industries = industries.iloc[1:]
industries

,industry
27,Advertising Agencies
73,Aerospace & Defense
77,Agricultural Inputs
3,Airlines
9,Airports & Air Services
...,...
30,Utilities—Regulated Electric
131,Utilities—Regulated Gas
68,Utilities—Regulated Water
24,Utilities—Renewable


In [ ]:
for row in industries.itertuples(index = False):
  cursor.execute("INSERT INTO industry (name) VALUES (%s)", row)
conn.commit()
print("|---------------| Data successfully written to database. |---------------|")

## Write industry_returns to SMD

```sql
CREATE TABLE IF NOT EXISTS `industry_return` (
  `id` INT(11) NOT NULL AUTO_INCREMENT,
  `industry_id` INT(11) NOT NULL,
  `return_date` DATE NOT NULL,
  `simple_return` DECIMAL(11,6) NULL DEFAULT NULL,
  `created_date` DATETIME NULL DEFAULT CURRENT_TIMESTAMP(),
  `last_updated` DATETIME NULL DEFAULT CURRENT_TIMESTAMP() ON UPDATE CURRENT_TIMESTAMP(),
  PRIMARY KEY (`id`),
  INDEX `return_date` (`return_date` ASC),
  INDEX `industry_id` (`industry_id` ASC),
  CONSTRAINT `fk_industry_id`
    FOREIGN KEY (`industry_id`)
    REFERENCES `industry` (`id`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION)
  ENGINE = InnoDB
  DEFAULT CHARACTER SET = utf8;
```

In [80]:
industries = pd.read_sql("SELECT DISTINCT(name) FROM industry;", conn)
industries = list(industries.name)
industries

['Advertising Agencies',
 'Aerospace & Defense',
 'Agricultural Inputs',
 'Airlines',
 'Airports & Air Services',
 'Aluminum',
 'Apparel Manufacturing',
 'Apparel Retail',
 'Asset Management',
 'Auto & Truck Dealerships',
 'Auto Manufacturers',
 'Auto Parts',
 'Banks—Diversified',
 'Banks—Regional',
 'Beverages—Brewers',
 'Beverages—Non-Alcoholic',
 'Beverages—Wineries & Distilleries',
 'Biotechnology',
 'Broadcasting',
 'Building Materials',
 'Building Products & Equipment',
 'Business Equipment & Supplies',
 'Capital Markets',
 'Chemicals',
 'Closed-End Fund - Equity',
 'Closed-End Fund - Foreign',
 'Coking Coal',
 'Communication Equipment',
 'Computer Hardware',
 'Confectioners',
 'Conglomerates',
 'Consulting Services',
 'Consumer Electronics',
 'Copper',
 'Credit Services',
 'Department Stores',
 'Diagnostics & Research',
 'Discount Stores',
 'Drug Manufacturers—General',
 'Drug Manufacturers—Specialty & Generic',
 'Education & Training Services',
 'Electrical Equipment & Parts',


In [81]:
count = 1

for industry in industries:
  industry_df = pd.DataFrame()
  sql = """
    SELECT ins.id, p.price_date, p.adj_close_price
    FROM instrument AS ins
    INNER JOIN price AS p
    ON ins.id = p.ticker_id
    WHERE industry = '{}';""".format(industry)

  tmp = pd.read_sql(sql, conn)
  tmp['simple_return'] = tmp.adj_close_price.pct_change()

  for i in range(1, len(tmp.id)):
    if tmp.id.at[i] != tmp.id.at[i-1]:
      tmp.simple_return.at[i] = None

  industry_df = tmp.groupby('price_date').mean().iloc[:,-1].to_frame()
  industry_df["price_date"] = industry_df.index
  industry_df.reset_index(drop = True, inplace = True)
  industry_df['id'] = count
  industry_df = industry_df.reindex(columns = ['id', 'price_date', 'simple_return'])
  industry_df.replace({np.nan: None}, inplace = True)

  for row in industry_df.itertuples(index = False):
    cursor.execute("INSERT INTO industry_return (industry_id, return_date, simple_return) VALUES (%s, %s, %s)", row)

  count += 1

conn.commit()
print("|---------------| Data successfully written to database. |---------------|")

|---------------| Data successfully written to database. |---------------|


In [20]:
industry_returns = pd.DataFrame()

# for i in range(1, len(industries) + 1):
for i in range(1, 5):
  tmp = pd.read_sql("SELECT return_date, simple_return FROM industry_return WHERE industry_id = {}".format(i), conn)
  # industry_returns = pd.concat([industry_returns, tmp], axis = 1)

tmp

,return_date,simple_return
0,1980-01-02,NaN
1,1980-01-03,-0.005741
2,1980-01-04,0.017334
3,1980-01-07,-0.022726
4,1980-01-08,-0.034881
...,...,...
10660,2022-04-11,0.010945
10661,2022-04-12,0.007821
10662,2022-04-13,0.043127
10663,2022-04-14,0.001510


In [5]:
tmp.set_index('industry_id').T

industry_id,1,1,1,1,1,1,1,1,1,1,...,151,151,151,151,151,151,151,151,151,151
return_date,1980-03-17,1980-03-18,1980-03-19,1980-03-20,1980-03-21,1980-03-24,1980-03-25,1980-03-26,1980-03-27,1980-03-28,...,2022-04-04,2022-04-05,2022-04-06,2022-04-07,2022-04-08,2022-04-11,2022-04-12,2022-04-13,2022-04-14,2022-04-18
simple_return,NaN,-0.005419,0.005889,0.004547,-0.003401,-0.029438,-0.003509,0.011333,-0.014908,0.00712,...,-0.002236,-0.004773,-0.016316,0.006289,-0.005986,-0.003661,-0.018964,0.010144,-0.007429,-0.00739
